![tracker](https://us-central1-vertex-ai-mlops-369716.cloudfunctions.net/pixel-tracking?path=statmike%2Fvertex-ai-mlops%2FDev%2Fnew&file=HOLDER.ipynb)
<!--- header table --->
<table align="left">
  <td style="text-align: center">
    <a href="https://colab.research.google.com/github/statmike/vertex-ai-mlops/blob/main/Dev/new/HOLDER.ipynb">
      <img src="https://cloud.google.com/ml-engine/images/colab-logo-32px.png" alt="Google Colaboratory logo">
      <br>Run in<br>Colab
    </a>
  </td>
  <td style="text-align: center">
    <a href="https://console.cloud.google.com/vertex-ai/colab/import/https%3A%2F%2Fraw.githubusercontent.com%2Fstatmike%2Fvertex-ai-mlops%2Fmain%2FDev%2Fnew%2FHOLDER.ipynb">
      <img width="32px" src="https://lh3.googleusercontent.com/JmcxdQi-qOpctIvWKgPtrzZdJJK-J3sWE1RsfjZNwshCFgE_9fULcNpuXYTilIR2hjwN" alt="Google Cloud Colab Enterprise logo">
      <br>Run in<br>Colab Enterprise
    </a>
  </td>      
  <td style="text-align: center">
    <a href="https://github.com/statmike/vertex-ai-mlops/blob/main/Dev/new/HOLDER.ipynb">
      <img src="https://cloud.google.com/ml-engine/images/github-logo-32px.png" alt="GitHub logo">
      <br>View on<br>GitHub
    </a>
  </td>
  <td style="text-align: center">
    <a href="https://console.cloud.google.com/vertex-ai/workbench/deploy-notebook?download_url=https://raw.githubusercontent.com/statmike/vertex-ai-mlops/main/Dev/new/HOLDER.ipynb">
      <img src="https://lh3.googleusercontent.com/UiNooY4LUgW_oTvpsNhPpQzsstV5W8F7rYgxgGBD85cWJoLmrOzhVs_ksK_vgx40SHs7jCqkTkCk=e14-rj-sc0xffffff-h130-w32" alt="Vertex AI logo">
      <br>Open in<br>Vertex AI Workbench
    </a>
  </td>
</table>

## A HOLDING PLACE FOR CONTENT IN THIS FOLDER

- goals:
    - data inputs
    - feature preprocess (before, at, inside)
    - post processing, and modularity
    - tensorboard
    - saving and reusing, show some modularity here
    - custom jogs
    - experiment tracking
    - model registry
    - evals in model registry
    - model monitoring for autoencoders
    
- dev:
    - autoencoder architecture
    - NAS
    - Anomaly detection with
    - contrastive learning
    - VAE
    - CVAE

### Training In TensorFlow

Build a normalization layer with [Keras Preprocessing Layers](https://www.tensorflow.org/guide/keras/preprocessing_layers). In this case all columns are already numeric so applying [tf.keras.layers.Normalization](https://www.tensorflow.org/api_docs/python/tf/keras/layers/Normalization) to the 'feature_array' column using the option `axis = -1` will calculate the mean and variance for each element of the 'feature_array' across all records.  This calculation only need to be done onece and can be triggered with the built in [.adapt()](https://www.tensorflow.org/api_docs/python/tf/keras/layers/Normalization#adapt) method.  The `adapt()` method forces the calculation of the mean and variance across and input dataset.  In the case, we modify the `tf.data.Dataset` reader built above to only return the 'feature_array' column and then pass it to the adapt method.

> Tip: apply a larger batch size to this reader to speed up performance.  The only calculations being made are the mean and variance and they are only computed once, prior to training.

In [43]:
feature_array_reader = training_reader.map(fn1).map(lambda v: v.pop('feature_array'))
normalizer = tf.keras.layers.Normalization(name = 'normalize', axis = -1)
normalizer.adapt(feature_array_reader.prefetch(2).batch(10000))
normalizer.mean, normalizer.variance

(<tf.Tensor: shape=(1, 30), dtype=float32, numpy=
 array([[ 9.4811133e+04, -2.1519326e-04,  3.1602196e-04, -5.2488595e-04,
          6.9466559e-04, -1.2641819e-03,  2.0892750e-03, -7.2106207e-04,
         -1.0636140e-03,  1.4059977e-03, -7.1558170e-05, -6.4140302e-04,
         -1.5961546e-03,  1.8235012e-03, -6.6740625e-04,  4.2201288e-04,
         -2.3144973e-04,  5.9940410e-04, -7.0123409e-04, -1.1209438e-03,
          7.4361498e-04, -5.4229691e-04,  7.6822005e-04,  3.2623837e-04,
          3.5052517e-04, -5.9398869e-04,  4.6557584e-04, -6.2941969e-04,
         -8.2514744e-05,  8.8535156e+01]], dtype=float32)>,
 <tf.Tensor: shape=(1, 30), dtype=float32, numpy=
 array([[2.2556255e+09, 3.8344266e+00, 2.7213738e+00, 2.3109169e+00,
         2.0030899e+00, 1.9093831e+00, 1.7799078e+00, 1.5511755e+00,
         1.4520742e+00, 1.2102603e+00, 1.1956733e+00, 1.0426062e+00,
         1.0046861e+00, 9.9211246e-01, 9.2476696e-01, 8.3907151e-01,
         7.6823246e-01, 7.2538882e-01, 7.0431167e-01,

Similarly, build a de-normalizer to help return final reconstructed values from the autoencoder to the original scale.  This also uses [tf.keras.layers.Normalization()](https://www.tensorflow.org/api_docs/python/tf/keras/layers/Normalization) but the mean and variance calculated for the normalization layer can be directly input while also using the `invert = True` argument to indicate an inverse transformation.

In [44]:
denormalizer = tf.keras.layers.Normalization(
    name = 'denormalize',
    mean = normalizer.mean,
    variance = normalizer.variance,
    invert = True
)

Build the model with layers:

In [45]:
# feature inputs: autoencoder
feature_inputs = [tf.keras.Input(shape = (1,), dtype = dtypes.float64, name = feature) for feature in training_data.columns]

# input layer of concatenated features
feature_layer = tf.keras.layers.Concatenate(name = 'feature_layer')(feature_inputs)

# use pre-learned normalizer a layer in model
norm_layer = normalizer(feature_layer)

# encoder
encoder = tf.keras.layers.Dense(128, activation = tf.nn.relu)(norm_layer)
encoder = tf.keras.layers.Dense(64, activation = tf.nn.relu)(encoder)
encoder = tf.keras.layers.Dense(8, activation = tf.nn.relu, name = 'encoder')(encoder)

# decoder
decoder = tf.keras.layers.Dense(64, activation = tf.nn.relu)(encoder)
decoder = tf.keras.layers.Dense(128, activation = tf.nn.relu)(decoder)
decoder = tf.keras.layers.Dense(feature_layer.shape[1], activation = tf.nn.sigmoid, name = 'decoder')(decoder)

# de-normalize 
reconstruct = denormalizer(decoder)

# define loss function - custom
def mae_loss(norm_layer, decoder):
    return tf.keras.losses.mae(norm_layer, decoder)

Create a model from the layers using [tf.keras.Model](https://www.tensorflow.org/api_docs/python/tf/keras/Model).  The inputs will be the first layer, 'feature_inputs' and the outputs will each of: encoder, decoder, and reconstruct layers.

In [46]:
model = tf.keras.Model(
    inputs = feature_inputs,
    outputs = {
        'feature_layer': feature_layer,
        'norm_layer': norm_layer,
        'decoder': decoder,
        'reconstruct': reconstruct
    },
    name = 'autoencoder_from_dataframe'
)

Compile the model to make it ready for training.  

In [47]:
model.compile(
    optimizer = tf.keras.optimizers.Adam(), #SGD or Adam
    loss = {'decoder': tf.keras.losses.MeanAbsoluteError()},
    #loss = {'decoder': mae_loss},
    metrics = {'decoder': [
        tf.keras.metrics.RootMeanSquaredError(name = 'rmse'),
        tf.keras.metrics.MeanSquaredError(name = 'mse'),
        tf.keras.metrics.MeanAbsoluteError(name = 'mae'),
        tf.keras.metrics.MeanSquaredLogarithmicError(name = 'msle'),
    ]}
)

In [48]:
model.fit(
    training_reader.prefetch(2).map(fn1).map(lambda v: (v, v.pop('feature_array'))).shuffle(1000).batch(100),
    epochs = 2
)

Epoch 1/2
2281/2281 [==============================] - 15s 6ms/step - loss: 3163.7671 - decoder_loss: 3163.7671 - decoder_rmse: 19360.3203 - decoder_mse: 374821568.0000 - decoder_mae: 3163.7671 - decoder_msle: 4.1063
Epoch 2/2
2281/2281 [==============================] - 13s 6ms/step - loss: 3163.7390 - decoder_loss: 3163.7390 - decoder_rmse: 19360.3203 - decoder_mse: 374822112.0000 - decoder_mae: 3163.7390 - decoder_msle: 4.0862


### Prediction

Retrieve a record, also called an instance, to use for prediction:

In [63]:
ds_iter = iter(training_reader.batch(1).take(1))
instance = {key: value.numpy() for key, value in next(ds_iter).items()}
instance

{'Time': array([2812]),
 'V1': array([-0.63340299]),
 'V2': array([0.96361604]),
 'V3': array([2.49494562]),
 'V4': array([2.09905099]),
 'V5': array([-0.40433067]),
 'V6': array([0.23586158]),
 'V7': array([-0.00793191]),
 'V8': array([0.21144152]),
 'V9': array([-0.20981682]),
 'V10': array([0.3082976]),
 'V11': array([-1.20499231]),
 'V12': array([-0.47470781]),
 'V13': array([-0.65406356]),
 'V14': array([-0.47459911]),
 'V15': array([-0.42841779]),
 'V16': array([0.53665148]),
 'V17': array([-0.38065462]),
 'V18': array([0.02865054]),
 'V19': array([-0.68796943]),
 'V20': array([-0.17498476]),
 'V21': array([0.01467553]),
 'V22': array([0.01627818]),
 'V23': array([-0.06146247]),
 'V24': array([0.35519634]),
 'V25': array([-0.1790855]),
 'V26': array([-0.10694743]),
 'V27': array([-0.21503926]),
 'V28': array([0.0506978]),
 'Amount': array([0.])}

Get the prediction for the instance directly from the model using the [predict()](https://www.tensorflow.org/api_docs/python/tf/keras/Model#predict) method:

In [64]:
prediction = model.predict(instance)
prediction

1/1 [==============================] - 0s 40ms/step


{'feature_layer': array([[ 2.8120000e+03, -6.3340300e-01,  9.6361601e-01,  2.4949455e+00,
          2.0990510e+00, -4.0433067e-01,  2.3586158e-01, -7.9319049e-03,
          2.1144152e-01, -2.0981681e-01,  3.0829760e-01, -1.2049923e+00,
         -4.7470781e-01, -6.5406358e-01, -4.7459912e-01, -4.2841780e-01,
          5.3665149e-01, -3.8065460e-01,  2.8650539e-02, -6.8796945e-01,
         -1.7498475e-01,  1.4675528e-02,  1.6278177e-02, -6.1462473e-02,
          3.5519636e-01, -1.7908551e-01, -1.0694742e-01, -2.1503925e-01,
          5.0697796e-02,  0.0000000e+00]], dtype=float32),
 'norm_layer': array([[-1.937092  , -0.32335705,  0.58393896,  1.6415733 ,  1.482617  ,
         -0.2916958 ,  0.17522429, -0.00578969,  0.17634982, -0.19200009,
          0.28201008, -1.1794863 , -0.472007  , -0.6584891 , -0.49283284,
         -0.4681614 ,  0.61253834, -0.44764012,  0.03497453, -0.8430712 ,
         -0.22730693,  0.02058318,  0.02136702, -0.10057119,  0.5855404 ,
         -0.34210312, -0.2228

The model has named outputs:

In [65]:
model.outputs

[<KerasTensor: shape=(None, 30) dtype=float32 (created by layer 'decoder')>,
 <KerasTensor: shape=(None, 30) dtype=float32 (created by layer 'feature_layer')>,
 <KerasTensor: shape=(None, 30) dtype=float32 (created by layer 'normalize')>,
 <KerasTensor: shape=(None, 30) dtype=float32 (created by layer 'denormalize')>]

Which can be directly referenced in the predictions:

In [66]:
prediction['reconstruct']

array([[ 1.4230456e+05, -2.1519324e-04,  1.4952000e+00,  1.5196446e+00,
         1.4121767e+00,  6.1348355e-03,  4.7503728e-01,  6.1340198e-02,
         1.1801603e-01,  4.9134474e-03,  9.5579021e-02,  4.7037434e-03,
         1.5275577e-01,  1.8235052e-03,  1.4947075e-04,  3.7509971e-03,
        -2.3144971e-04,  2.7462911e-02,  8.7620683e-02,  6.6268378e-01,
         6.3060969e-02,  2.1521749e-02,  1.3648245e-01,  3.7682979e-04,
         1.6728850e-01, -5.6953978e-04,  4.6557657e-04,  2.9199759e-02,
         2.1776773e-02,  3.3989557e+02]], dtype=float32)

Multiple records, also called instances, can be predicted at the same time:

In [67]:
ds_iter = iter(training_reader.batch(2).take(1))
instances = {key: value.numpy() for key, value in next(ds_iter).items()}
instances

{'Time': array([2812, 3150]),
 'V1': array([-0.63340299,  1.31328087]),
 'V2': array([ 0.96361604, -0.25792282]),
 'V3': array([2.49494562, 0.11846283]),
 'V4': array([ 2.09905099, -0.73555665]),
 'V5': array([-0.40433067, -0.56930772]),
 'V6': array([ 0.23586158, -0.73357721]),
 'V7': array([-0.00793191, -0.13865918]),
 'V8': array([ 0.21144152, -0.14164134]),
 'V9': array([-0.20981682,  1.70801916]),
 'V10': array([ 0.3082976 , -1.10329377]),
 'V11': array([-1.20499231, -1.08782009]),
 'V12': array([-0.47470781,  0.64467588]),
 'V13': array([-0.65406356, -0.21536864]),
 'V14': array([-0.47459911, -0.07471497]),
 'V15': array([-0.42841779,  0.28787333]),
 'V16': array([ 0.53665148, -1.00176397]),
 'V17': array([-0.38065462,  0.09376776]),
 'V18': array([ 0.02865054, -0.07254906]),
 'V19': array([-0.68796943,  1.1083853 ]),
 'V20': array([-0.17498476, -0.14514383]),
 'V21': array([ 0.01467553, -0.08246737]),
 'V22': array([0.01627818, 0.12606591]),
 'V23': array([-0.06146247, -0.223157

In [68]:
prediction = model.predict(instances)
prediction['reconstruct']

1/1 [==============================] - 0s 41ms/step


array([[ 1.4230456e+05, -2.1519324e-04,  1.4951999e+00,  1.5196446e+00,
         1.4121767e+00,  6.1348290e-03,  4.7503763e-01,  6.1340131e-02,
         1.1801631e-01,  4.9134521e-03,  9.5578760e-02,  4.7037387e-03,
         1.5275575e-01,  1.8235052e-03,  1.4947308e-04,  3.7510036e-03,
        -2.3144971e-04,  2.7462857e-02,  8.7620832e-02,  6.6268361e-01,
         6.3060932e-02,  2.1521749e-02,  1.3648215e-01,  3.7682979e-04,
         1.6728833e-01, -5.6953978e-04,  4.6557657e-04,  2.9199719e-02,
         2.1776769e-02,  3.3989557e+02],
       [ 1.4230456e+05,  1.9579538e+00,  3.1602365e-04,  1.6818026e-01,
         5.4540168e-02, -1.2403572e-03,  2.9143826e-03,  4.6561495e-04,
         2.7090572e-02,  1.0978662e+00, -7.0910319e-05, -3.6981062e-04,
         6.6006875e-01,  1.8235403e-03,  2.9768083e-02,  3.6315274e-01,
        -2.3123699e-04,  2.2773892e-02,  8.3688293e-03, -1.1208523e-03,
         7.4997096e-04,  3.4153730e-02,  1.8506378e-01,  3.2702286e-04,
         1.5659017e-02,

The input side of the model was built to take named inputs, columns.  An advantage of this is that the inputs at prediction time can change order and the model will reassemble them in the correct order for inference.  Had the model been trained on an array of feature values, then the array would need to be provide here at inference time in the exact same order.  This can be very helpful when inputs are images with nature order of features: rows, columns, and layers of pixels.  For tabular data arranged in named columns, using named inputs prevent training/serving skew by removing the need to both gather the input columns and order them.

To demonstrate, the single instance above will be pass in the correct and in reversed order:

In [69]:
instance

{'Time': array([2812]),
 'V1': array([-0.63340299]),
 'V2': array([0.96361604]),
 'V3': array([2.49494562]),
 'V4': array([2.09905099]),
 'V5': array([-0.40433067]),
 'V6': array([0.23586158]),
 'V7': array([-0.00793191]),
 'V8': array([0.21144152]),
 'V9': array([-0.20981682]),
 'V10': array([0.3082976]),
 'V11': array([-1.20499231]),
 'V12': array([-0.47470781]),
 'V13': array([-0.65406356]),
 'V14': array([-0.47459911]),
 'V15': array([-0.42841779]),
 'V16': array([0.53665148]),
 'V17': array([-0.38065462]),
 'V18': array([0.02865054]),
 'V19': array([-0.68796943]),
 'V20': array([-0.17498476]),
 'V21': array([0.01467553]),
 'V22': array([0.01627818]),
 'V23': array([-0.06146247]),
 'V24': array([0.35519634]),
 'V25': array([-0.1790855]),
 'V26': array([-0.10694743]),
 'V27': array([-0.21503926]),
 'V28': array([0.0506978]),
 'Amount': array([0.])}

In [73]:
instance_reversed = {k:v for k,v in reversed(instance.items())}
instance_reversed

{'Amount': array([0.]),
 'V28': array([0.0506978]),
 'V27': array([-0.21503926]),
 'V26': array([-0.10694743]),
 'V25': array([-0.1790855]),
 'V24': array([0.35519634]),
 'V23': array([-0.06146247]),
 'V22': array([0.01627818]),
 'V21': array([0.01467553]),
 'V20': array([-0.17498476]),
 'V19': array([-0.68796943]),
 'V18': array([0.02865054]),
 'V17': array([-0.38065462]),
 'V16': array([0.53665148]),
 'V15': array([-0.42841779]),
 'V14': array([-0.47459911]),
 'V13': array([-0.65406356]),
 'V12': array([-0.47470781]),
 'V11': array([-1.20499231]),
 'V10': array([0.3082976]),
 'V9': array([-0.20981682]),
 'V8': array([0.21144152]),
 'V7': array([-0.00793191]),
 'V6': array([0.23586158]),
 'V5': array([-0.40433067]),
 'V4': array([2.09905099]),
 'V3': array([2.49494562]),
 'V2': array([0.96361604]),
 'V1': array([-0.63340299]),
 'Time': array([2812])}

In [75]:
prediction = model.predict(instance)['reconstruct']
prediction_reversed = model.predict(instance_reversed)['reconstruct']
prediction, prediction_reversed

1/1 [==============================] - 0s 41ms/step


(array([[ 1.4230456e+05, -2.1519324e-04,  1.4952000e+00,  1.5196446e+00,
          1.4121767e+00,  6.1348355e-03,  4.7503728e-01,  6.1340198e-02,
          1.1801603e-01,  4.9134474e-03,  9.5579021e-02,  4.7037434e-03,
          1.5275577e-01,  1.8235052e-03,  1.4947075e-04,  3.7509971e-03,
         -2.3144971e-04,  2.7462911e-02,  8.7620683e-02,  6.6268378e-01,
          6.3060969e-02,  2.1521749e-02,  1.3648245e-01,  3.7682979e-04,
          1.6728850e-01, -5.6953978e-04,  4.6557657e-04,  2.9199759e-02,
          2.1776773e-02,  3.3989557e+02]], dtype=float32),
 array([[ 1.4230456e+05, -2.1519324e-04,  1.4952000e+00,  1.5196446e+00,
          1.4121767e+00,  6.1348355e-03,  4.7503728e-01,  6.1340198e-02,
          1.1801603e-01,  4.9134474e-03,  9.5579021e-02,  4.7037434e-03,
          1.5275577e-01,  1.8235052e-03,  1.4947075e-04,  3.7509971e-03,
         -2.3144971e-04,  2.7462911e-02,  8.7620683e-02,  6.6268378e-01,
          6.3060969e-02,  2.1521749e-02,  1.3648245e-01,  3.76829

Both prediction look identical even thought the instance was input in a different order.  This can also be programatically checked for confirmation:

In [76]:
(prediction == prediction_reversed).all()

True

### Post-processing

An incredible feature of TensorFlow/Keras is being able to treat models as layers. 

The trained model above outputs the reconstructed feature array from the trained autoencoder.  It would be great to have more information and different formatting though. Like:
- report instance level metrics to help interpret the reconstruction: MAE, MSE, MSLE
    - base these on the actual values and the normalized values
- return the reconstructed feature array to named elements matching the named inputs to the model
- order the reconstructed feature array in decending order by magnitude of the error
    - base magnitude of error on the absolute difference in normalized values
    
The following builds post-processing model that takes the outputs of the autoencoder as inputs.  Then, a new model is built that combines these together into a single model.

In [155]:
# metric calcs on denormalized values
mean_absolute_error = tf.keras.losses.mae(reconstruct, feature_layer)
mean_squared_error = tf.keras.losses.mse(reconstruct, feature_layer)
mean_squared_log_error = tf.keras.losses.msle(reconstruct, feature_layer)

# metric calc on normalized values
norm_mean_absolute_error = tf.keras.losses.mae(norm_layer, decoder)
norm_mean_squared_error = tf.keras.losses.mse(norm_layer, decoder)
norm_mean_squared_log_error = tf.keras.losses.msle(norm_layer, decoder)

# list reconstruction error for each feature
errors = [{feature_inputs[v].name : val} for v, val in enumerate(reconstruct[0,:])]

# errors ordered by norm error absolute magnitude
norm_abs_diffs = tf.math.abs(norm_layer - decoder)
ordered_norm_abs_diffs = tf.argsort(norm_abs_diffs, direction = 'DESCENDING')
errors_impact_order = [i for i in ordered_norm_abs_diffs[0,:]]

In [156]:
post_model = tf.keras.Model(
    #inputs = {k: v for k, v in model.output.items() if k in ['reconstruct', 'feature_layer']},
    inputs = model.output,
    outputs = {
        'mean_absolute_error': mean_absolute_error[0],
        'mean_squared_error': mean_squared_error[0],
        'mean_squared_log_error': mean_squared_log_error[0],
        'norm_mean_absolute_error': norm_mean_absolute_error[0],
        'norm_mean_squared_error': norm_mean_squared_error[0],
        'norm_mean_squared_log_error': norm_mean_squared_log_error[0],
        'errors': errors,
        'errors_impact_order': errors_impact_order
    },
    name = 'autoencoder_post'
)

In [157]:
full_model = tf.keras.Model(
    inputs = model.inputs,
    outputs = post_model(model(model.inputs))
)

In [167]:
full_model.predict(instance)

1/1 [==============================] - 0s 45ms/step


{'mean_absolute_error': 4661.44775390625,
 'mean_squared_error': 648609664.0,
 'mean_squared_log_error': 1.67367684841156,
 'norm_mean_absolute_error': 0.5154851078987122,
 'norm_mean_squared_error': 0.6112011671066284,
 'norm_mean_squared_log_error': 0.06236143782734871,
 'errors': [{'Time': 142304.5625},
  {'V1': -0.00021519324218388647},
  {'V2': 1.4952000379562378},
  {'V3': 1.5196446180343628},
  {'V4': 1.4121767282485962},
  {'V5': 0.006134835537523031},
  {'V6': 0.47503727674484253},
  {'V7': 0.061340197920799255},
  {'V8': 0.11801602691411972},
  {'V9': 0.0049134474247694016},
  {'V10': 0.09557902067899704},
  {'V11': 0.0047037433832883835},
  {'V12': 0.1527557671070099},
  {'V13': 0.0018235051538795233},
  {'V14': 0.000149470753967762},
  {'V15': 0.003750997129827738},
  {'V16': -0.0002314497105544433},
  {'V17': 0.0274629108607769},
  {'V18': 0.08762068301439285},
  {'V19': 0.6626837849617004},
  {'V20': 0.06306096911430359},
  {'V21': 0.02152174897491932},
  {'V22': 0.136482